# Problem Titanica
## konkretniej którzy pasażerowie przeżyli
Mając dane dotyczące ok. 2/3 pasażerów Titanica oraz informację czy przeżyli wypadek, program decyduje czy pozostałe ok. 1/3 przeżyło. Podział danych na trenujące i testujące został już za nas wykonany, znajdują się w osobnych plikach.  
Pasażerowie mogą jedynie przeżyć (1) lub nie (0), jest to zatem problem klasyfikacji


### ta sekcja kodu importuje potrzebne biblioteki do obróbki danych ###

In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#wypisuje dostepne pliki
import os
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


./kaggle/input\gender_submission.csv
./kaggle/input\test.csv
./kaggle/input\train.csv


### dane trenujace oraz testujace sa ladowane z plikow csv do odpowiednich zmiennych oraz podgladane jest ich 5 pierwszych wierszy

In [2]:
train_data = pd.read_csv("./kaggle/input/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("./kaggle/input/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### Określić którzy z pasażerow przeżyli (jak klasyfikować - przeżyli czy nie)  
#### co osiągniemy poprzez nauczenie go rozpoznawania cech, które najpewniej oznaczają że pasażer przeżył, na bazie znanych danych trenujacych  
w tym wypadku jest to dokonywane przy pomocy różnych modeli machine learning z biblioteki sklearn i wykorzystania tylko niektórych znanych cech o pasażerach, którzy przeżyli lub nie  
dodatkowo aby tego dokonać konieczne było użycie metod radzących sobie z brakującymi danymi

### wykorzystane modele to:
##### Tree - drzewo decyzyjne cech  
##### RandomForestClassifier - model testujący wiele wariacji drzewa  
##### ExtraTreesClassifier - model testujący wiele wariacji drzewa z dodatkowymi elementami losowości  
##### SGDClasssifier - Stochastic Gradient Descent - model działający na gradientach  
##### Support Vector Machines  - wykorzystują reprezentację jako punkty w przestrzeni i sieci
##### MLPClassifier - sieć neuronona złożona z warstw perceptronów
##### Lasso - stosuje tzw. coordinate descent  
##### KNearestNeighbors - szuka "sąsiadów" punktów o podobnych cechach

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsClassifier

from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer(strategy = 'most_frequent')



y = train_data["Survived"] #nazwa jest wymogiem competition

#wybranie interesujacych cech
features = ["Pclass", "Sex", "Age", "Parch", "Fare", "SibSp"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])
X_imputed = my_imputer.fit_transform(X)
X_test_imputed = my_imputer.fit_transform(X_test)


#utworzenie modelu RandomForest
model = RandomForestClassifier(n_estimators = 100, max_depth =6, random_state = 5)
#ograniczenie ilosci estymatorow i maksymalnej ma sens, gdyz jest to dosyc maly zbior
#jest zatem zagrozenie zjawiska overfit
model.fit(X_imputed,y)
predictions = model.predict(X_test_imputed)

#ponownie, format outputu jest wymogiem competition
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission1.csv', index=False)
print("udalo sie")

#utworzenie i przetestowanie modelu SGDClassifier
modelSGD = SGDClassifier(loss="hinge", penalty="l2", max_iter=50)
modelSGD.fit(X_imputed,y)
predictionsSGD = modelSGD.predict(X_test_imputed)
outputSGD = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictionsSGD})
outputSGD.to_csv('submission2.csv', index=False)
print("udalo sie SGD")

#utworzenie i przetestowanie modelu Support Vector Machine
modelSVM = svm.SVC() #C poniewaz Classifier
modelSVM.fit(X_imputed,y)
predictionsSVM = modelSVM.predict(X_test_imputed)
outputSVM = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictionsSVM})
outputSVM.to_csv('submission3.csv', index=False)
print("udalo sie SVM")

#pojedyncze drzewo decyzyjne
modelTree = tree.DecisionTreeClassifier(max_depth = 6)
modelTree.fit(X_imputed,y)
predictionsTree = modelTree.predict(X_test_imputed)
outputTree = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictionsTree})
outputTree.to_csv('submission4.csv', index=False)
print("udalo sie tree")

#extremalnie losowe drzewo
modelExtra= ExtraTreesClassifier(n_estimators=100, max_depth=None)
modelExtra.fit(X_imputed,y)
predictionsExtra = modelExtra.predict(X_test_imputed)
outputExtra = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictionsExtra})
outputExtra.to_csv('submission5.csv', index=False)
print("udalo sie extra")


#Siec neuronowa Multi Layer Perceptron
modelMLP= MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(6,6,2))
modelMLP.fit(X_imputed,y)
predictionsMLP = modelMLP.predict(X_test_imputed)
outputMLP = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictionsMLP})
outputMLP.to_csv('submission6.csv', index=False)
print("udalo sie MLP")

#Inna wersja MLP, wiecej warstw, niebezpieczenstwo overfit
modelMLP2= MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(6,5,4,3,2))
modelMLP2.fit(X_imputed,y)
predictionsMLP2 = modelMLP2.predict(X_test_imputed)
outputMLP2 = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictionsMLP2})
outputMLP2.to_csv('submission7.csv', index=False)
print("udalo sie MLP2")

#Lasso
modelLasso= Lasso(alpha=0.2)
modelLasso.fit(X_imputed,y)
predictionsLasso = modelLasso.predict(X_test_imputed)
outputLasso = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictionsLasso})
outputLasso.to_csv('submission8.csv', index=False)
print("udalo sie Lasso")

#KNeighbors
modelNeigh= KNeighborsClassifier(n_neighbors=5)
modelNeigh.fit(X_imputed,y)
predictionsNeigh = modelNeigh.predict(X_test_imputed)
outputNeigh = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictionsNeigh})
outputNeigh.to_csv('submission9.csv', index=False)
print("udalo sie Neigh")

udalo sie
udalo sie SGD
udalo sie SVM
udalo sie tree
udalo sie extra
udalo sie MLP
udalo sie MLP2
udalo sie Lasso
udalo sie Neigh


C:\Users\User\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


### Uwagi
#### metoda get_dummies jest jedną z metod utworzenia matryc o odpowiednim formacie zawierających jedynie wskazane w liście features kolumny oraz wykonujaca metode hot encoding zastepujaca wartosci kategoryczne (opisowe) liczbami  
  
dodatkowo na kolumnach z brakujacymi wartosciami zostalo dokonane tzw. imputation, czyli uzupelnienie ich, gdyz zawieraja w wiekszosci wartosciowe informacje, poprzez umieszczenie w nich wartości najczęściej występujących


### Wnioski
Pojedyńcze drzewo decyzyjne poradziło sobie gorzej od lasu [Random Forest] (spodziewalnie, gdyż las rozpatruje więcej możliwości i wybiera najlepszą z nich)
od obu poradziły sobie gorzej zarówno SVM oraz SGD, nie są to modele przystosowane do problemu gdzie wiele zmiennych jest opisowych (nawet przy użyciu one hot encoding), a raczej do zmiennych liczbowych o zakresach większych niż binarne 0-1

Użycie zbyt wielu cech może pogorszyć model - będzie on uwzględniał cechy, które przypadkiem dla danych trenujących dawały konkretne rezultaty klasyfikacji, jednakże w rzeczywistości (na danych testujących) nie muszą tego oznaczać

Las (Random Forest) używa wielu różnych losowych kombinacji kolejności cech, a następnie porównuje wyniki, co rozpatruje wiele możliwości bardziej lub mniej znaczących cech

w przypadku lasu zwiekszanie ilosci estymatorow do pewnego momentu zwiekszalo poprawnosc wynikow, ale tylko do pewnego momentu

extra losowy las czyni wybieranie cech prawdziwie losowym
poradził sobie nieco gorzej niż randomforest

MLP mimo bycia być może przesadnym narzędziem do zadania o tak małej ilości danych ,poradził sobie dobrze z 3 warstwami, choć gorzej od RandomForest

model Lasso powinien być w stanie sprowadzając zadanie do problemu regresji poradzić sobie również z klasyfikacją, okazał się to jednak model nieodpowiedni do typu zadania

zgodnie z przypuszczeniami dla 5 warstw model MLP osiąga gorsze wyniki, zachodzi najpewniej zjawisko **overfit**

Dla wszystkich modeli związanych z drzewami mała ilość estymatorów oznaczała gorsze wyniki, gdyż najprawdpodobniej zachodziło zjawisko **underfit** - niewystarczające dopasowanie

### Ostateczny najlepszy osiągnięty wynik:
#### 0.78708
#### Co niestety jest dalekie od ideału